Currently using code taken from
https://gist.github.com/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e#file-music_genre_classification-ipynb
for exploration of possible solutions 

# Music genre classification notebook

## Importing Libraries

In [31]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

## Extracting music and features

### Dataset

We use [GTZAN genre collection](http://marsyasweb.appspot.com/download/data_sets/) dataset for classification. 
<br>
<br>
The dataset consists of 10 genres i.e
 * Blues
 * Classical
 * Country
 * Disco
 * Hiphop
 * Jazz
 * Metal
 * Pop
 * Reggae
 * Rock
 
Each genre contains 100 songs. Total dataset: 1000 songs

## Extracting the Spectrogram for every Audio

In [33]:
cmap = plt.get_cmap('binary')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'../../audio/testfiles/GTZAN/genres/{g}'):
        songname = f'../../audio/testfiles/GTZAN/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
 

<Figure size 720x720 with 0 Axes>

All the audio files get converted into their respective spectrograms .WE can noe easily extract features from them.

## Extracting features from Spectrogram


We will extract

* Mel-frequency cepstral coefficients (MFCC)(20 in number)
* Spectral Centroid,
* Zero Crossing Rate
* Chroma Frequencies
* Spectral Roll-off.

In [3]:
header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Writing data to csv file

We write the data to a csv file 

In [4]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
filepath = '../../audio/testfiles/GTZAN/genres/'
for g in genres:
    for filename in os.listdir(f'{filepath}/{g}'):
        songname = f'{filepath}/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        #rmse = librosa.feature.rmse(y=y, S=None, frame_length=2048, hop_length=512, center=True, pad_mode='reflect')
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        #to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

# Analysing the Data in Pandas

In [5]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00085.wav,0.315363,1312.308199,1673.915613,2638.117381,0.059416,-180.123596,131.420259,0.566188,43.152929,...,2.718054,0.772589,1.997588,-6.093858,3.484042,-8.341479,3.204648,-0.926944,-2.243686,blues
1,blues.00051.wav,0.393756,1977.172377,1927.803692,3942.834492,0.106627,-55.579243,114.935848,-37.052831,64.896513,...,12.782317,-16.528681,3.793788,-7.890870,8.477609,-4.065210,3.207441,-5.178250,-1.279524,blues
2,blues.00038.wav,0.265883,1513.422107,2140.606779,3449.679140,0.044378,-192.667187,111.190774,21.372723,26.398341,...,-5.044727,-17.631820,-7.921687,-15.152404,-12.342546,-17.227765,-7.771728,-13.033961,-17.945750,blues
3,blues.00069.wav,0.291884,2371.099278,2209.699346,5004.111407,0.125141,-121.408845,96.106080,-19.613255,48.011654,...,6.793616,-19.333882,-0.114447,-12.250433,-4.927438,-10.409581,-2.049566,-1.768211,-3.962159,blues
4,blues.00049.wav,0.277484,1318.656822,1904.761177,3046.681577,0.039973,-255.965774,110.823046,21.505215,67.571233,...,-2.442929,-7.130865,-11.163917,-12.609408,2.742009,-8.948849,-7.940728,-2.344157,-0.705606,blues


In [6]:
data.shape

(1000, 27)

In [7]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

## Encoding the Labels

In [8]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

## Scaling the Feature columns

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

## Dividing data into training and Testing set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
len(y_train)

800

In [12]:
len(y_test)

200

In [13]:
X_train[10]

array([-0.19983702, -1.12235633, -0.80675114, -0.98918641, -1.31311229,
       -0.80344064,  0.76971867, -0.03839601,  1.21870608,  0.94871863,
        0.50510579, -0.51532196,  0.52044259, -0.15009777,  0.63851434,
       -0.43717433,  0.47832901, -0.85517149, -0.40676076, -0.28863192,
       -0.86616406, -0.09871817,  0.2717664 , -0.72311185, -0.64936228])

# Classification with Keras

## Building our Network

In [14]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train,
                    y_train,
                    epochs=200,
                    batch_size=128)
                   

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
800/800 [==============================] - 0s 467us/step - loss: 2.2002 - acc: 0.2337
Epoch 2/200
800/800 [==============================] - 0s 31us/step - loss: 1.8906 - acc: 0.3488
Epoch 3/200
800/800 [==============================] - 0s 30us/step - loss: 1.6508 - acc: 0.4200
Epoch 4/200
800/800 [==============================] - 0s 29us/step - loss: 1.4649 - acc: 0.4738
Epoch 5/200
800/800 [==============================] - 0s 32us/step - loss: 1.3206 - acc: 0.5413
Epoch 6/200
800/800 [==============================] - 0s 33us/step - loss: 1.2174 - acc: 0.5875
Epoch 7/200
800/800 [==============================] - 0s 32us/step - loss: 1.1268 - acc: 0.6112
Epoch 8/200
800/800 [==============================] - 0s 28us/step - loss: 1.0544 - acc: 0.6550
Epoch 9/200
800/800 [==============================] - 0s 31us/step - loss: 0.9912 - acc: 0.6687
Epoch 10/200
800/800 [==============================] - 0s 31us/step - loss: 0

800/800 [==============================] - 0s 28us/step - loss: 0.0030 - acc: 1.0000
Epoch 165/200
800/800 [==============================] - 0s 28us/step - loss: 0.0029 - acc: 1.0000
Epoch 166/200
800/800 [==============================] - 0s 30us/step - loss: 0.0029 - acc: 1.0000
Epoch 167/200
800/800 [==============================] - 0s 33us/step - loss: 0.0028 - acc: 1.0000
Epoch 168/200
800/800 [==============================] - 0s 31us/step - loss: 0.0028 - acc: 1.0000
Epoch 169/200
800/800 [==============================] - 0s 34us/step - loss: 0.0027 - acc: 1.0000
Epoch 170/200
800/800 [==============================] - 0s 33us/step - loss: 0.0027 - acc: 1.0000
Epoch 171/200
800/800 [==============================] - 0s 32us/step - loss: 0.0026 - acc: 1.0000
Epoch 172/200
800/800 [==============================] - 0s 34us/step - loss: 0.0026 - acc: 1.0000
Epoch 173/200
800/800 [==============================] - 0s 29us/step - loss: 0.0026 - acc: 1.0000
Epoch 174/200
800/800 [=

In [17]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 192us/step


In [18]:
print('test_acc: ',test_acc)

test_acc:  0.64


Tes accuracy is less than training dataa accuracy. This hints at Overfitting

## Validating our approach
Let's set apart 200 samples in our training data to use as a validation set:

In [32]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]
x_norm = preprocessing.scale(x_val)
x_norm

array([[ 2.7604279 ,  0.80956891, -0.13702939, ...,  0.66305118,
        -0.89615763,  0.91367764],
       [ 1.06577848,  1.09246308,  0.22651024, ...,  1.17337777,
        -0.83382548,  2.535132  ],
       [ 0.19108424,  1.00271175,  1.02043055, ...,  0.13309015,
         0.12838833,  0.03948205],
       ...,
       [ 0.28626474, -0.00950845, -0.08346922, ...,  0.17329929,
         0.27462279, -0.42498359],
       [ 0.20785118,  0.32865694,  0.2287461 , ..., -0.31187917,
        -0.13889837,  0.72759639],
       [-0.10703749,  0.59479979,  0.29674725, ...,  0.77058291,
        -0.70099519,  1.40212598]])

Now let's train our network for 20 epochs:

In [20]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=200,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/200
600/600 [==============================] - 0s 581us/step - loss: 2.2731 - acc: 0.1533 - val_loss: 2.1121 - val_acc: 0.3400
Epoch 2/200
600/600 [==============================] - 0s 44us/step - loss: 2.1232 - acc: 0.3300 - val_loss: 1.9617 - val_acc: 0.3650
Epoch 3/200
600/600 [==============================] - 0s 42us/step - loss: 1.9855 - acc: 0.3433 - val_loss: 1.8428 - val_acc: 0.3650
Epoch 4/200
600/600 [==============================] - 0s 46us/step - loss: 1.8531 - acc: 0.3617 - val_loss: 1.7407 - val_acc: 0.4050
Epoch 5/200
600/600 [==============================] - 0s 42us/step - loss: 1.7313 - acc: 0.3933 - val_loss: 1.6479 - val_acc: 0.4500
Epoch 6/200
600/600 [==============================] - 0s 45us/step - loss: 1.6093 - acc: 0.4367 - val_loss: 1.5531 - val_acc: 0.4700
Epoch 7/200
600/600 [==============================] - 0s 42us/step - loss: 1.4963 - acc: 0.5183 - val_loss: 1.4411 - val_acc: 0.4900
Epoch 8/200
600

600/600 [==============================] - 0s 50us/step - loss: 0.0100 - acc: 1.0000 - val_loss: 1.9302 - val_acc: 0.6350
Epoch 122/200
600/600 [==============================] - 0s 44us/step - loss: 0.0107 - acc: 1.0000 - val_loss: 1.9371 - val_acc: 0.6450
Epoch 123/200
600/600 [==============================] - 0s 42us/step - loss: 0.0105 - acc: 1.0000 - val_loss: 1.9000 - val_acc: 0.6450
Epoch 124/200
600/600 [==============================] - 0s 53us/step - loss: 0.0097 - acc: 1.0000 - val_loss: 1.8696 - val_acc: 0.6450
Epoch 125/200
600/600 [==============================] - 0s 53us/step - loss: 0.0093 - acc: 1.0000 - val_loss: 1.8659 - val_acc: 0.6450
Epoch 126/200
600/600 [==============================] - 0s 44us/step - loss: 0.0091 - acc: 1.0000 - val_loss: 1.8887 - val_acc: 0.6450
Epoch 127/200
600/600 [==============================] - 0s 50us/step - loss: 0.0090 - acc: 1.0000 - val_loss: 1.9213 - val_acc: 0.6500
Epoch 128/200
600/600 [==============================] - 0s 45

In [21]:
results

[2.341206932067871, 0.605]

## Predictions on Test Data

In [22]:
predictions = model.predict(X_test)

In [23]:
predictions[0].shape

(10,)

In [24]:
np.sum(predictions[0])

1.0

In [25]:
np.argmax(predictions[0])

6